# Cryptographie - principes de base

## 0- Principes

Le but de la cryptographie est de cacher un texte donné de tel manière que seul son destinataire légitime puisse le lire.
Les manières de cacher ce texte sont multiples. On s'intéresse ici à une transformation du texte pour le rendre illisible.

En pratique pour nous, le texte à cacher est une chaîne de caractères s. On cherche donc à calculer une chaîne de caractère tirée de s de telle manière que notre destinataire (avec qui on peut éventuellement partager un secret, la <b>clé</b>), et avec un peu de chance seul lui, puisse déchiffrer ce texte.

### Cadre

On se limite à des textes contenant uniquement des caractères en minuscule et des espaces, pour simplifier les raisonnements. Dans le texte de référence utilisé, tous les autres caractères seront remplacés par des espaces.

### Texte à cacher

On fixe pour tout le cours un texte à crypter : les histoires extraordinaires d'Edgar Poe, traduites par Charles Baudelaire et disponible librement sur le site du projet Gutenberg ici : http://www.gutenberg.org/ebooks/20761
Le fichier texte, légèrement nettoyé (majuscules enlevé, préface enlevée) est dans le dossier contenant ce notebook et peut donc être ouvert directement par Python :

In [1]:
with open('HistoiresExtraordinaires.txt') as f:
    texte = f.read()
    texte = texte.replace('\r', '')
print(texte[:2500]) #on regarde si le texte a bien été importé en affichant le début

edgar allan poe

histoires extraordinaires

traduction par charles baudelaire premiere publication en france en 1856

double assassinat dans la rue morgue

quelle chanson chantaient les sirenes? quel nom achille avait-il pris,
quand il se cachait parmi les femmes?-questions embarrassantes, il est
vrai, mais qui ne sont pas situees au dela de toute conjecture.

sir thomas browne.


les facultes de l'esprit qu'on definit par le terme _analytiques_ sont
en elles-memes fort peu susceptibles d'analyse. nous ne les apprecions
que par leurs resultats. ce que nous en savons, entre autres choses,
c'est qu'elles sont pour celui qui les possede a un degre extraordinaire
une source de jouissances des plus vives. de meme que l'homme fort se
rejouit dans son aptitude physique, se complaÃ®t dans les exercices qui
provoquent les muscles a l'action, de meme l'analyse prend sa gloire
dans cette activite spirituelle dont la fonction est de debrouiller. il
tire du plaisir meme des plus triviales occasions

## 1- Chiffrement de César

### 1.1 - Description

Le principe est assez simple :
<ul>
<li>on donne un numéro à chaque lettre de l'alphabet</li>
<li>la clé de chiffrage est un nombre entre 1 et le nombre de lettre - 1</li>
<li>la chaîne cryptée est obtenue en décalant tous les caractères de clé dans l'alphabet, étant entendu que l'on revient au début à la fin de la liste...</li>
</ul>

In [2]:
LETTRES = 'abcdefghijklmnopqrstuvwxyz '

def indice(lettre):
    """
    Retourne l'indice de la lettre donnée dans la chaîne LETTRES
    indice('a') retourne 0, indice('z') retourne 25
    indice(' ') retourne 26, indice(n'importe quoi d'autre) retourne 26
    """
    if 97<=ord(lettre)<=122:
        return ord(lettre)-ord("a")
    else:
        return 26

In [3]:
def cesar(chaine, cle):
    #s est notre chaine codée qu'on construit au fur et à mesure
    s = ''
    n = len(LETTRES)
    for i in range(len(chaine)):
        #on décale la lettre chaine[i] de clé. On fait % n pour revenir à 0 si on dépasse 26
        s = s + LETTRES[(indice(chaine[i]) + cle) % n]
    return s

In [4]:
cesar(texte, 14)[:500]

'sruoenozzoancbsnnvwfgbwesfnskgeoberwaowesfnngeorhqgwbancoenqvoezsfnpohrszowesnces wsesnchpzwqogwbansanteoaqsnsannnnnnnrbhpzsnoffoffwaognroafnzonehsn beuhsnndhszzsnqvoafbanqvoagowsagnzsfnfwesasfnndhsznab noqvwzzsnoiowgnwzncewfnndhoarnwznfsnqoqvowgncoe wnzsfnts  sfnndhsfgwbafns poeeoffoagsfnnwznsfgnieownn owfndhwnasnfbagncofnfwghssfnohnrszonrsngbhgsnqbaxsqghesnnnfwengvb ofnpebjasnnnnzsfntoqhzgsfnrsnznsfcewgndhnbanrstwawgncoenzsngse snnoaozlgwdhsfnnfbagnsanszzsfn s sfntbegncshnfhfqscgwpzsfnrnoaozlf'

### 1.2 - Retrouver le texte d'origine

Le principe pour notre correspondant est assez simple, il suffit de décaler dans l'autre sens.

In [5]:
#on crypte
messageCode=cesar('hello world',3)
print(messageCode)
#on décrypte
messageDecode=cesar(messageCode,-3)
print(messageDecode)

khoorczruog
hello world


In [6]:
#modulo 27, faire +14 ou -13 revient au même
cesar(cesar('hello world', 13), 14)

'hello world'

In [7]:
#bien sûr, avec notre texte qui contient des caractères autres que des minuscules
#on perd un peu d'information qui se transforme en espace
cesar(cesar(texte, 4), -4)[:500]

'edgar allan poe  histoires extraordinaires  traduction par charles baudelaire premiere publication en france en       double assassinat dans la rue morgue  quelle chanson chantaient les sirenes  quel nom achille avait il pris  quand il se cachait parmi les femmes  questions embarrassantes  il est vrai  mais qui ne sont pas situees au dela de toute conjecture   sir thomas browne    les facultes de l esprit qu on definit par le terme  analytiques  sont en elles memes fort peu susceptibles d analys'

### 1.3 - Cryptanalyse

Il est très facile de retrouver la clé secrète du chiffrement de césar si on connait l'alphabet utilisé.
Ainsi un espion, le gouvernement, ou toute autre personne satisfaisant votre sens de la théorie du complot, peut retrouver le message en clair.

La méthode est simple : force brute. On essaie toutes les clés possible et on regarde à la main les messages obtenus. il y a tellement peu de clés possible que la cryptanalyse est quasiment immédiate.

In [8]:
import random
#on choisit une clé aléatoire qu'on n'affiche pas pour la démonstration
cle_choisie = random.randint(1, len(LETTRES) - 1)

message = 'ceci est un message un tout petit peu plus long'
crypte = cesar(message, cle_choisie)

for i in range(1, len(LETTRES)):
    print('clé : ', i, ' message décodé : ' + cesar(crypte, -i))

clé :  1  message décodé : dfdjaftuavoanfttbhfavoaupvuaqfujuaqfvaqmvtampoh
clé :  2  message décodé : ceci est un message un tout petit peu plus long
clé :  3  message décodé : bdbhzdrsztmzldrr fdztmzsntszodshszodtzoktrzknmf
clé :  4  message décodé : acagycqryslykcqqzecyslyrmsryncrgryncsynjsqyjmle
clé :  5  message décodé :  b fxbpqxrkxjbppydbxrkxqlrqxmbqfqxmbrxmirpxilkd
clé :  6  message décodé : zazewaopwqjwiaooxcawqjwpkqpwlapepwlaqwlhqowhkjc
clé :  7  message décodé : y ydv novpivh nnwb vpivojpovk odovk pvkgpnvgjib
clé :  8  message décodé : xzxcuzmnuohugzmmvazuohunionujzncnujzoujfomufiha
clé :  9  message décodé : wywbtylmtngtfyllu ytngtmhnmtiymbmtiyntienltehg 
clé :  10  message décodé : vxvasxklsmfsexkktzxsmfslgmlshxlalshxmshdmksdgfz
clé :  11  message décodé : uwu rwjkrlerdwjjsywrlerkflkrgwk krgwlrgcljrcfey
clé :  12  message décodé : tvtzqvijqkdqcviirxvqkdqjekjqfvjzjqfvkqfbkiqbedx
clé :  13  message décodé : susypuhipjcpbuhhqwupjcpidjipeuiyipeujpeajhpadcw
clé :  14  message dé

Pas trop difficile de deviner ce qu'avait choisi python comme clé...

## 2 - Chiffrement par permutation

### 2.1 - Principe

Au lieu de bêtement décaler chaque lettre, on choisit une bijection de l'alphabet dans lui même, c'est à dire que chaque lettre est transformée en une autre.

En pratique une bijection est représentée par une chaîne de caractère ou une liste possédant les même lettres que LETTRES mais dans le désordre

In [9]:
def permute(chaine, bijection):
    s = ''
    n = len(LETTRES)
    for c in chaine:
        s = s + bijection[indice(c)]
    return s

In [10]:
#ou on peut faire une permutation très simple en inversant simplement les "o" et les "e"
bij1='abcdofghijklmnepqrstuvwxyz '
print(permute('hello world', bij1))
#ou une permutation plus complexe car complètement aléatoire
bij = list(LETTRES)
random.shuffle(bij)
print(permute('hello world', bij))

holle werld
tmwwclvc wy


### 2.2 - Déchiffrement

Comment retrouver le message d'origine connaissant le message chiffré et la permutation ? Facile, il suffit de calculer la réciproque, c'est à dire trouver la position du caractère courant dans bijection.

In [11]:
def decode_permute(chiffre, bijection):
    s = ''
    n = len(LETTRES)
    for c in chiffre:
        s = s + LETTRES[bijection.index(c)]
    return s

In [12]:
#on crypte
messageCode=permute('hello world', bij)
print(messageCode)
#on décrypte
messageDecode=decode_permute(messageCode,bij)
print(messageDecode)

tmwwclvc wy
hello world


Cette fois le nombre de clé est plutôt vaste : le "a" peut se transformer en 26 lettres différentes, le "b" en 25 (puisque ça ne peut pas être la même que "a"), le "c" en 24, etc. Cela fait donc $26 \times 25 \times 24 \times ... \times 2 \times 1 = 26! \approx 3 \times 10^{29}$ possibilités de clés. Si on pouvait lire 1 000 000 de textes à la seconde (pour vérifier si c'est le bon), il faudrait quand même un million de milliard d'années pour arriver au bout...

On ne peut donc pas attaquer ce genre de code par de la force brute.

### 2.3 - Cryptanalyse

Idée : la probabilité d'apparition des lettres dans un texte en français n'est pas du tout uniforme. On doit pouvoir identifier les voyelles et les consones courantes assez rapidement. On peut aussi utiliser le fait que certaines lettres sont souvent doublées en français, comme le l ou le t ou le p ou encore que de nombreux mots se terminent par s.

In [13]:
def frequence(chaine):
    """
    Calcule le nombre d'occurences de chaque lettre dans la chaine donnée.
    """
    #on crée un liste de compteurs initialisés à 0
    occ = [ 0 for c in LETTRES]
    
    #on incrémente le compteur de la lettre rencontrée dans la chaine
    for c in chaine:
        i = indice(c)
        occ[i] = occ[i] + 1
    
    #on modifie les compteurs en transformant les effectifs en fréquences
    total = max(len(chaine), 1)
    for i in range(len(LETTRES)):
        occ[i] = 100 * occ[i] / total
    
    #on affiche les résultats triés par ordre de fréquence en affichant la fréquence et la lettre
    resultat = [(occ[i], LETTRES[i]) for i in range(len(LETTRES))]
    resultat.sort()
    return resultat

In [14]:
#on suppose que le texte est codée par une bijection secrète IL FAUT FAIRE COMME SI ON NE CONNAISSAIT PAS CETTE BIJECTION!!!
bij=['u', 'm', 'k', 'i', 'q', 'y', 'l', 'c', 'p', 'a', 'j', ' ', 't', 'n', 'x', 'f', 'r', 'g', 'v', 'o', 'e', 's', 'h', 'w', 'z', 'd', 'b']
messageCode = permute(texte, bij)
frequence(messageCode)

[(0.014544483114300344, 'h'),
 (0.018700049718386156, 'j'),
 (0.09617168426598595, 'd'),
 (0.2175735943424929, 'z'),
 (0.30973812509739607, 'w'),
 (0.5532840107154253, 'a'),
 (0.5972142862443325, 'c'),
 (0.7140153904377444, 'l'),
 (0.7649210813377956, 'm'),
 (0.8524848062096038, 'y'),
 (0.9982264635386133, 'r'),
 (1.2453342633887161, 's'),
 (2.3333506481941835, 'f'),
 (2.4538620797126725, 'k'),
 (2.5825362313463294, 't'),
 (2.953717376946994, 'i'),
 (4.106590283394801, 'x'),
 (4.17189204431615, ' '),
 (4.928353579352771, 'g'),
 (5.047826119220238, 'e'),
 (5.692681008318554, 'p'),
 (5.819425789743171, 'o'),
 (5.826104378928309, 'n'),
 (6.231568949012682, 'v'),
 (6.477637857211763, 'u'),
 (13.84842570811597, 'q'),
 (21.143819707774618, 'b')]

Il suffit maintenant d'aller sur [wikipedia](https://fr.wikipedia.org/wiki/Fr%C3%A9quence_d%27apparition_des_lettres_en_fran%C3%A7ais#Fr%C3%A9quence_des_caract%C3%A8res_dans_le_corpus_de_Wikip%C3%A9dia_en_fran%C3%A7ais) ou tout autre site pour trouver une table référence des fréquence de lettres dans les textes en français. 
On peut deviner plusieurs choses :
<ul>
<li>l'espace correspond au 21,14%</li>
<li>le e doit correspondre à la lettre la plus fréquente</li>
</ul>
Pour trouver des chiffres plus proches des tables données, il faut considérer des textes sans espaces. Allons-y

In [15]:
messageCode2 = messageCode.replace('b', '')
frequence(messageCode2)

[(0.0, 'b'),
 (0.018444316045237507, 'h'),
 (0.023714120629591082, 'j'),
 (0.12195833466646842, 'd'),
 (0.2759119114522264, 'z'),
 (0.3927886488409253, 'w'),
 (0.7016368389453616, 'a'),
 (0.7573462016942422, 'c'),
 (0.9054653519758944, 'l'),
 (0.9700204581342257, 'm'),
 (1.0810627690188188, 'y'),
 (1.265882344084362, 'r'),
 (1.5792475095468157, 's'),
 (2.958995274114532, 'f'),
 (3.1118196070607853, 'k'),
 (3.274995341869162, 't'),
 (3.7457018156358868, 'i'),
 (5.207696173180835, 'x'),
 (5.29050738807782, ' '),
 (6.249800029736755, 'g'),
 (6.40130691153692, 'e'),
 (7.219067658644644, 'p'),
 (7.379796698467428, 'o'),
 (7.38826602726371, 'n'),
 (7.902448388851352, 'v'),
 (8.214496103167717, 'u'),
 (17.561623777358285, 'q')]

Les chiffres obtenus sont proches de ceux de la table de référence. Essayons de décrypter.

On va créer une fonction qui permute deux lettres données :

In [16]:
def echange(chaine, c1, c2):
    s = ''
    for c in chaine:
        #si on rencontre c1, on remplace par c2
        if c == c1:
            s += c2
        #si on rencontre c2, on remplace par c2
        elif c == c2:
            s += c1
        #sinon on garde le même caractère
        else:
            s += c
    return s

In [17]:
c2 = echange(messageCode, 'b', ' ');
c2 = echange(c2, 'q', 'e')
c2 = echange(c2, 'v', 'a')
c2 = echange(c2, 'u', 'i')
c2 = echange(c2, 'o', 's')
c2 = echange(c2, 'n', 'n')
c2 = echange(c2, 'p', 'r')
c2 = echange(c2, 'g', 't')

c2[:5000]

'eulit ibbin fxe  crasxrtea ewstixturnirtea  stiuqksrxn fit kcitbea miquebirte ftegrete fqmbrkisrxn en ytinke en       uxqmbe iaaiaarnis uina bi tqe gxtlqe  pqebbe kcinaxn kcinsirens bea artenea  pqeb nxg ikcrbbe ioirs rb ftra  pqinu rb ae kikcirs fitgr bea yeggea  pqeasrxna egmittiaainsea  rb eas otir  gira pqr ne axns fia arsqeea iq uebi ue sxqse kxnveksqte   art scxgia mtxhne    bea yikqbsea ue b eaftrs pq xn ueyrnrs fit be setge  inibzsrpqea  axns en ebbea gegea yxts feq aqakefsrmbea u inibzae  nxqa ne bea ifftekrxna pqe fit beqta teaqbsisa  ke pqe nxqa en aioxna  enste iqstea kcxaea  k eas pq ebbea axns fxqt kebqr pqr bea fxaaeue i qn uelte ewstixturnirte qne axqtke ue vxqraainkea uea fbqa oroea  ue gege pqe b cxgge yxts ae tevxqrs uina axn ifsrsque fczarpqe  ae kxgfbi  s uina bea ewetkrkea pqr ftxoxpqens bea gqakbea i b iksrxn  ue gege b inibzae ftenu ai lbxrte uina kesse iksrorse afrtrsqebbe uxns bi yxnksrxn eas ue uemtxqrbbet  rb srte uq fbirart gege uea fbqa stroribea xkkiarxn

Beaucoup de mots se terminent par a, ce sont sûrement des s. On voit aussi le 2eme mot avec un double b, on parie sur un double l à la place :

In [18]:
c2 = echange(c2, 'a', 's')
c2 = echange(c2, 'b', 'l')

c2[0:5000]

'eubit illin fxe  crsaxrtes ewatixturnirtes  atiuqkarxn fit kcitles miquelirte ftegrete fqmlrkiarxn en ytinke en       uxqmle ississrnia uins li tqe gxtbqe  pqelle kcinsxn kcinairena les srtenes  pqel nxg ikcrlle ioira rl ftrs  pqinu rl se kikcira fitgr les yegges  pqesarxns egmittissinaes  rl esa otir  girs pqr ne sxna fis sraqees iq ueli ue axqae kxnvekaqte   srt acxgis mtxhne    les yikqlaes ue l esftra pq xn ueyrnra fit le aetge  inilzarpqes  sxna en elles geges yxta feq sqskefarmles u inilzse  nxqs ne les ifftekrxns pqe fit leqts tesqlaias  ke pqe nxqs en sioxns  enate iqates kcxses  k esa pq elles sxna fxqt kelqr pqr les fxsseue i qn uebte ewatixturnirte qne sxqtke ue vxqrssinkes ues flqs oroes  ue gege pqe l cxgge yxta se tevxqra uins sxn ifaraque fczsrpqe  se kxgfli  a uins les ewetkrkes pqr ftxoxpqena les gqskles i l ikarxn  ue gege l inilzse ftenu si blxrte uins keaae ikarorae sfrtraqelle uxna li yxnkarxn esa ue uemtxqrllet  rl arte uq flirsrt gege ues flqs atroriles xkkisrxn

Prochaine étape, utiliser les espaces de manière plus systématiques, pour réutiliser l'idée précédente. Par exemple, trouver les mots de longueur 1 et 2.

In [19]:
mots = c2.split()

long_1 = [m for m in mots if len(m)== 1]
mots_1 = set() #ensemble : ne peut pas contenir deux fois le même élément
for m in long_1:
    mots_1.add((m, long_1.count(m)))
print(mots_1)

long_2 = [m for m in mots if len(m) == 2]
mots_2 = set() #ensemble : ne peut pas contenir deux fois le même élément
for m in long_2:
    mots_2.add((m, long_2.count(m)))
print(mots_2)

{('f', 99), ('a', 360), ('i', 2870), ('n', 726), ('r', 8), ('u', 1834), ('m', 50), ('d', 1), ('z', 366), ('e', 95), ('s', 493), ('y', 86), ('v', 569), ('l', 2267), ('x', 5), ('q', 4), ('w', 3), ('j', 1), ('p', 3), ('o', 47), ('t', 44), ('b', 20), ('k', 491), ('g', 486), ('c', 32), ('h', 3)}
{('yt', 2), ('rr', 1), ('ae', 117), ('li', 3285), ('kc', 21), ('ml', 1), ('ea', 3237), ('at', 1), ('le', 2389), ('qu', 5), ('ne', 794), ('xq', 485), ('yl', 4), ('xy', 4), ('ke', 715), ('ee', 1), ('iu', 1), ('kl', 1), ('xc', 32), ('mz', 2), ('gr', 1), ('ue', 5233), ('ki', 15), ('eq', 49), ('fq', 72), ('gz', 1), ('en', 1247), ('lq', 6), ('ox', 7), ('kr', 43), ('nx', 1), ('kt', 29), ('tr', 1), ('qy', 1), ('se', 527), ('uq', 980), ('in', 5), ('ro', 1), ('xs', 2), ('oi', 13), ('nr', 79), ('gi', 279), ('oq', 46), ('kx', 15), ('io', 1), ('qs', 1), ('rl', 1689), ('ft', 3), ('ry', 1), ('ve', 1768), ('im', 17), ('xt', 66), ('te', 5), ('si', 451), ('fl', 2), ('bx', 6), ('pq', 770), ('et', 1), ('qt', 38), ('bt'

Le but est de trouver a, l, d qui sont très commun en mots d'une lettre comme en mots de deux ("l'", "d'", "la", "le"...)

On observe que le mot "ue" apparait 5249 fois, comme on est assez sûr du e et sans doute du l, on peut parier que le u se transforme en d.

De même, on on observe que "le" et "li" apparaissent environ 3000 fois, donc on parie que ce sont les mots le et la. On va donc inverser le i et le a.    

In [20]:
c3 = echange(c2, 'u', 'd')
c3 = echange(c2, 'i', 'a')
c3[0:5000]

'eubat allan fxe  crsixrtes ewitaxturnartes  itauqkirxn fat kcatles maquelarte ftegrete fqmlrkairxn en ytanke en       uxqmle assassrnai uans la tqe gxtbqe  pqelle kcansxn kcaniareni les srtenes  pqel nxg akcrlle aoari rl ftrs  pqanu rl se kakcari fatgr les yegges  pqesirxns egmattassanies  rl esi otar  gars pqr ne sxni fas sriqees aq uela ue ixqie kxnvekiqte   srt icxgas mtxhne    les yakqlies ue l esftri pq xn ueyrnri fat le ietge  analzirpqes  sxni en elles geges yxti feq sqskefirmles u analzse  nxqs ne les afftekrxns pqe fat leqts tesqliais  ke pqe nxqs en saoxns  enite aqites kcxses  k esi pq elles sxni fxqt kelqr pqr les fxsseue a qn uebte ewitaxturnarte qne sxqtke ue vxqrssankes ues flqs oroes  ue gege pqe l cxgge yxti se tevxqri uans sxn afirique fczsrpqe  se kxgfla  i uans les ewetkrkes pqr ftxoxpqeni les gqskles a l akirxn  ue gege l analzse ftenu sa blxrte uans keiie akirorie sfrtriqelle uxni la yxnkirxn esi ue uemtxqrllet  rl irte uq flarsrt gege ues flqs itrorales xkkasrxn

On voit "elles sxni" qui ressemblent à "elles sont".
On va tenter les permutations x<->o et i<->t.

In [21]:
c3 = echange(c3, 'x', 'o')
c3 = echange(c3, "i", "t")
c3[:5000]

'eubai allan foe  crstories ewtiaoiurnaries  tiauqktron fai kcailes maquelarie fiegreie fqmlrkatron en yianke en       uoqmle assassrnat uans la iqe goibqe  pqelle kcanson kcantarent les srienes  pqel nog akcrlle axart rl firs  pqanu rl se kakcart faigr les yegges  pqestrons egmaiiassantes  rl est xiar  gars pqr ne sont fas srtqees aq uela ue toqte konvektqie   sri tcogas miohne    les yakqltes ue l esfirt pq on ueyrnrt fai le teige  analztrpqes  sont en elles geges yoit feq sqskeftrmles u analzse  noqs ne les affiekrons pqe fai leqis iesqltats  ke pqe noqs en saxons  entie aqties kcoses  k est pq elles sont foqi kelqr pqr les fosseue a qn uebie ewtiaoiurnarie qne soqike ue voqrssankes ues flqs xrxes  ue gege pqe l cogge yoit se ievoqrt uans son aftrtque fczsrpqe  se kogfla  t uans les eweikrkes pqr fioxopqent les gqskles a l aktron  ue gege l analzse fienu sa blorie uans kette aktrxrte sfrirtqelle uont la yonktron est ue uemioqrllei  rl trie uq flarsri gege ues flqs tirxrales okkasron

In [22]:
c3 = echange(c3, 'z', 'y')
c3 = echange(c3, "i", "r")
c3[:5000]

'eubar allan foe  cistoires ewtraoruinaires  trauqktion far kcarles maquelaire fregiere fqmlikation en zranke en       uoqmle assassinat uans la rqe gorbqe  pqelle kcanson kcantaient les sirenes  pqel nog akcille axait il fris  pqanu il se kakcait fargi les zegges  pqestions egmarrassantes  il est xrai  gais pqi ne sont fas sitqees aq uela ue toqte konvektqre   sir tcogas mrohne    les zakqltes ue l esfrit pq on uezinit far le terge  analytipqes  sont en elles geges zort feq sqskeftimles u analyse  noqs ne les affrekions pqe far leqrs resqltats  ke pqe noqs en saxons  entre aqtres kcoses  k est pq elles sont foqr kelqi pqi les fosseue a qn uebre ewtraoruinaire qne soqrke ue voqissankes ues flqs xixes  ue gege pqe l cogge zort se revoqit uans son aftitque fcysipqe  se kogfla  t uans les ewerkikes pqi froxopqent les gqskles a l aktion  ue gege l analyse frenu sa bloire uans kette aktixite sfiritqelle uont la zonktion est ue uemroqiller  il tire uq flaisir gege ues flqs trixiales okkasion

In [23]:
c3 = echange(c3, 'c', 'h')
c3 = echange(c3, "x", "w")
c3 = echange(c3, "u", "d")
c3[:5000]

'edbar allan foe  histoires extraordinaires  tradqktion far kharles maqdelaire fregiere fqmlikation en zranke en       doqmle assassinat dans la rqe gorbqe  pqelle khanson khantaient les sirenes  pqel nog akhille await il fris  pqand il se kakhait fargi les zegges  pqestions egmarrassantes  il est wrai  gais pqi ne sont fas sitqees aq dela de toqte konvektqre   sir thogas mrocne    les zakqltes de l esfrit pq on dezinit far le terge  analytipqes  sont en elles geges zort feq sqskeftimles d analyse  noqs ne les affrekions pqe far leqrs resqltats  ke pqe noqs en sawons  entre aqtres khoses  k est pq elles sont foqr kelqi pqi les fossede a qn debre extraordinaire qne soqrke de voqissankes des flqs wiwes  de gege pqe l hogge zort se revoqit dans son aftitqde fhysipqe  se kogfla  t dans les exerkikes pqi frowopqent les gqskles a l aktion  de gege l analyse frend sa bloire dans kette aktiwite sfiritqelle dont la zonktion est de demroqiller  il tire dq flaisir gege des flqs triwiales okkasion

et ainsi de suite...

La décodage allant, on commence à comprendre le sens et pouvoir déchiffrer plus facilement.

Pour faciliter la lecture, il faudrait afficher différemment les lettres déjà trouvées. On y est presque. Le reste est à finir en exercice.

## 3- Vigénère

### 3.1 - Principe

La principale faiblesse du codage par substitution est que tous les "e" deviennnent une seule et même lettre dans le message codé et qu'une analyse statistique permet d'identifier par quoi elle a été substituée.
Le code de Vigénère apporte une soution à cette faiblesse car il introduit l'idée d'une substitution polyalphabétique, c'est à dire qu'une lettre donnée ne sera pas toujours transformée en la même lettre.

Au lieu d'utiliser un décalage uniforme sur tout le texte comme dans le code de César, on choisit un mot clé qui sert à indiquer de combien on se décale. 
<ul>
<li>On transforme le mot clé en liste de chiffres, qui seront nos décalages (a->+0, b->+1, c->+2,...), par exemple, si la clé est bac, alors la liste des décalages est de (1,0,2) (a->+0, b->+1, c->+2,...)</li>
<li>la première lettre du texte est décalée grâce au premier chiffre de la clé</li>
<li>la deuxième lettre du texte est décalée grâce au deuxième chiffre de la clé</li>
<li>on poursuit ainsi, en reprennant la clé depuis le début quand on l'a épuisée</li>
</ul>

La vidéo suivante, à regarder, explique le cryptage par substitution et de Vigénère : https://youtu.be/PIw_nuWsOFU

In [24]:
def vigenere(texte, mot_cle):
    liste_cle = [indice(c) for c in mot_cle]
    s = ''
    long_cle = len(liste_cle)
    long_alph = len(LETTRES)
    for i in range(len(texte)):
        s = s + LETTRES[(indice(texte[i]) + liste_cle[i % long_cle]) % long_alph]
    return s

In [25]:
vigenere('ceci est un test', 'motcle')

'osvkkidgswydeskv'

In [26]:
def dechiffre_vigenere(texte, mot_cle):
    liste_cle = [-indice(c) for c in mot_cle]
    s = ''
    long_cle = len(liste_cle)
    long_alph = len(LETTRES)
    for i in range(len(texte)):
        s = s + LETTRES[(indice(texte[i]) + liste_cle[i % long_cle]) % long_alph]
    return s

In [27]:
dechiffre_vigenere('osvkkidgswydeskv', 'motcle')

'ceci est un test'

### 3.2 - Cryptanalyse

Elle est beaucoup plus délicate que la précédente. L'idée est de trouver d'abord la longueur de la clé en procédant à des mesures d'incidences. Ensuite, on peut procéder à une analyse de fréquence pour déterminer chaque décalage.
La page Bibmath consacrée à la cryptanalyse de Vigénère détaille assez clairement ce procédé : http://www.bibmath.net/crypto/index.php?action=affiche&quoi=poly/viganalyse

On va regarder un exemple pour montrer le procédé :

In [28]:
#on code notre texte avec une clé de 5 caractères. ON SUPPOSE QU'ON NE CONNAIT PAS CETTE CLÉ
cle= "alien"
messageCode=vigenere(texte, cle)
messageCode=messageCode[:5000] #on se restreint à 5000 caractères pour ne pas surcharger le processeur 
print(messageCode)

eooed ltpnnkxsr kpmetzqvrskmafrlwvqiyimdechdfrllypttwrmplzdphlzprskjegdptevrphtdexqidekxyoltkefizvdrnknvnnnmdrnkhdm khhaumtimac eestvef oire widduphqarrbim abiylphguay s  npe tlqi tktie cqvrnp dmqempmnzudncsqpyekiznidhmy  zme kyynnohmy cmdpanpevtkxedmthprsknizmp dmqemwfizvwmexjedrl wnndmwm ttdrsdhzdathdzat dcuthrr cwrf  iwmstayrecheg ompn omdfoeaimczvnrcdbvr khwvrkalaml dorzdrr khdyechjncetxrsklimlkmwbrtadcukwrmdpnm idhtnrktimtpzqr kirnliamcup dmszvxmeyhiylp dzexmwmfzzxmppbdeuckibttjprskldnnltbeekhrauchrr wmwma xvrctwre abimplzdyeezwmrp yytlawm nmdcuphrauchi  cizanchdrndzimaeavrskklasp dmckmwf abdrlwmwmszvxmpzbvmcptyv abmmlp dboc iqekidgnklitrphijtbisddtvevrphy ek sgrnmdqekrsgic e cp dqechtyuchzvvp dmdphqrmphugektduoxuimfzzxmsphvrjzbmf oire cwrma amfuomdbhi mcuphdeekkszpwidmtkle sktie peidctkie abmmpbwzaqemrf wmwmme gyechemlkigfizvdmdphqrmphpmayipksphtdeyldeakopaibmdqay dpedaimanamhidmdeptzmfuptpr owrf widsoykxvoyhietklimdpjvauttprrkhmy dqvr obdbllqwvrkuizeklie  tye dzmhiltie zkgnstwre

On commence par lister tous les trigrammes existant dans ce texte et on les met dans un dicionnaire. La valeur associée dans le dictionnaire correspond à l'index de sa position dans le texte.

In [29]:
#on crée le dictionnaire vide
dictTrigrammes={}

#on parcourt le message codé au complet jusqu'à l'avant dernier caractère
for i in range(len(messageCode)-2):
    if messageCode[i:i+3] not in dictTrigrammes: #si le trigramme n'a jamais été rencontré
        dictTrigrammes[messageCode[i:i+3]]=[i]   #on crée une liste des positions de ce trigramme initialisée à [i]
    else:
        dictTrigrammes[messageCode[i:i+3]].append(i) #si le trigramme a déjà été rencontré, on ajoute sa nouvelle position
print(dictTrigrammes)

{'eoo': [0, 2253], 'ooe': [1], 'oed': [2], 'ed ': [3, 1718, 2208], 'd l': [4], ' lt': [5], 'ltp': [6], 'tpn': [7], 'pnn': [8], 'nnk': [9], 'nkx': [10, 4520], 'kxs': [11], 'xsr': [12], 'sr ': [13], 'r k': [14, 359, 379, 434, 1489, 1694, 1839, 2619, 4714], ' kp': [15], 'kpm': [16, 1071], 'pme': [17, 1268], 'met': [18], 'etz': [19], 'tzq': [20], 'zqv': [21], 'qvr': [22, 187, 952, 3782, 4542, 4712], 'vrs': [23, 583, 2588, 2748], 'rsk': [24, 64, 254, 394, 489, 584, 1014, 1069, 2309, 2399, 2629, 2749, 3164, 3389, 3579, 4634, 4684, 4914], 'skm': [25, 1675, 2350, 2400, 4915], 'kma': [26, 2196], 'maf': [27], 'afr': [28], 'frl': [29, 44, 1894], 'rlw': [30, 604, 1299, 3564], 'lwv': [31], 'wvq': [32], 'vqi': [33], 'qiy': [34, 3796], 'iyi': [35], 'yim': [36], 'imd': [37, 933, 1108, 1408, 3278, 3438, 4823], 'mde': [38, 897, 1368, 1827, 1883, 3423, 3637], 'dec': [39, 1219], 'ech': [40, 330, 385, 695, 830, 1045, 1425, 1805, 2410, 2650, 2710, 2970, 3095, 3105, 3115, 3130, 3450, 3590, 3930, 4890, 4920],

On veut connaitre l'écart qu'il y a entre les trigrammes identiques. On crée donc le dictionnaire dictTrigrammesEcart qui recense l'écart entre chacun des trigrammes.

In [30]:
dictTrigrammesEcart=dict()
for (key,value) in dictTrigrammes.items():
    dictTrigrammesEcart[key]=[value[i+1]-value[i] for i in range(len(value)-1)]
print(dictTrigrammesEcart)


{'eoo': [2253], 'ooe': [], 'oed': [], 'ed ': [1715, 490], 'd l': [], ' lt': [], 'ltp': [], 'tpn': [], 'pnn': [], 'nnk': [], 'nkx': [4510], 'kxs': [], 'xsr': [], 'sr ': [], 'r k': [345, 20, 55, 1055, 205, 145, 780, 2095], ' kp': [], 'kpm': [1055], 'pme': [1251], 'met': [], 'etz': [], 'tzq': [], 'zqv': [], 'qvr': [165, 765, 2830, 760, 170], 'vrs': [560, 2005, 160], 'rsk': [40, 190, 140, 95, 95, 430, 55, 1240, 90, 230, 120, 415, 225, 190, 1055, 50, 230], 'skm': [1650, 675, 50, 2515], 'kma': [2170], 'maf': [], 'afr': [], 'frl': [15, 1850], 'rlw': [574, 695, 2265], 'lwv': [], 'wvq': [], 'vqi': [], 'qiy': [3762], 'iyi': [], 'yim': [], 'imd': [896, 175, 300, 1870, 160, 1385], 'mde': [859, 471, 459, 56, 1540, 214], 'dec': [1180], 'ech': [290, 55, 310, 135, 215, 380, 380, 605, 240, 60, 260, 125, 10, 10, 15, 320, 140, 340, 960, 30], 'chd': [530, 855, 370, 485, 130, 555, 270, 1350], 'hdf': [3045], 'dfr': [1850], 'rll': [], 'lly': [], 'lyp': [], 'ypt': [], 'ptt': [2115], 'ttw': [1280, 1705], 'twr'

L'idée en regardant ce dictionnaire est de dire qu'un trigramme réapparait quand ce sont les trois même lettres qui ont été codées par les mêmes lettres de la clé. Cela veut donc dire que les écarts trouvés entre les trigrammes sont des multiples de la longueur de la clé.

Ici par exemple, presque tous les écarts semblent être des multiples de 5, ce qui nous amène à poser l'hypothèse que la clé est de longueur 5.

Tous les 5 caractères, le décalage est donc le même. Il faut donc trouver quel décalage a été fait sur les caractères en position 0,5,10,15... puis de trouver quel décalage a été fait sur les caractères en position 1,6,11,16..., puis ceux en position 2,7,12,17 et ainsi de suite.
Pour cela on va créer une fonction <code>sousTexte(texte,longueur,pos)</code> (à faire en exercice)

In [31]:
def sousTexte(texte,longueur,pos):
    #renvoit les caractères de texte situés aux positions pos, pos+longueur, pos+longueur*2, pos+longueur*3...
    #par exemple sousTexte("abcdefghijkl",2,0) renvoit "acegik" (on commence à 0 et on saute de 2 en 2)
    #par exemple sousTexte("abcdefghijkl",2,1) renvoit "bdfhjl" (on commence à 1 et on saute de 2 en 2)
    #par exemple sousTexte("abcdefghijkl",3,1) renvoit "behk"  (on commence à 1 et on saute de 3 en 3)   
    

SyntaxError: unexpected EOF while parsing (<ipython-input-31-22e132ed092e>, line 6)

On peut désormais analyser les fréquences des sous-textes. L'analyse des fréquences permettra de deviner le décalage.

Par exemple, si mon sous-texte contient une très grande fréquence de "g", alors je peux supposer que la lettre "e" a été décalée en "g" et que le décalage est donc de +2. Le caractère de la clé à cette position est donc c (a->+0, b->+1, c->+2,...) 

In [ ]:
sousTexte0=sousTexte(messageCode,5,0) #on récupère le sous-texte composé des caractères 0,5,10,15...
print(frequence(sousTexte))           #on observe quelle lettre est la plus fréquente, c'était sûrement un "e" dans le message en clair

Le reste de la cryptanalyse de Vigénère est à terminer en exercice. Vous allez donc implémenter la fonction <code>sousTexte()</code> et utiliser les analyses de fréquences pour en déduire la clé utilisée.